In [1]:
from apiclient.discovery import build
from apiclient.errors import HttpError
from oauth2client.tools import argparser
import urllib.request 
from urllib.parse import quote, unquote
import json 
import pandas as pd 
from pandas.io.json import json_normalize 
import pymysql as db
import re
import requests
import time

In [3]:
#servicekey = 'uwANzKRxZtNiDrBaW7T7cN%2FDykCM7m6%2BNmGtAhQ88yszsHuqb9%2BZeME%2FaKo3LP6%2Bpmu%2BIjVNPbF5WY9yv8QCow%3D%3D'
api_key_decode = 'uwANzKRxZtNiDrBaW7T7cN%2FDykCM7m6%2BNmGtAhQ88yszsHuqb9%2BZeME%2FaKo3LP6%2Bpmu%2BIjVNPbF5WY9yv8QCow%3D%3D'

In [22]:
local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
con = db.connect(
    host=local,
    user='admin',
    db='Nexpot',
    password='qwertyuiop123',
    charset='utf8mb4'
)
cur = con.cursor()

#cur.execute("select distinct places.placeID, places.pname, youtube.region from places,youtube where youtube.videoID = places.videoID and tour.title is NULL;")
cur.execute("select * from places where placeID not in (select tour.placeID from places, tour where places.placeID = tour.placeID);")
#cur.execute("select * from tour where placeID = '10821532'")
#cur.execute("select places.placeID, places.pname, youtube.region from youtube, places, tour where youtube.videoID = places.videoID and places.placeID = tour.placeID and tour.title is NULL;")
row_places = cur.fetchall()
#print(row_places)
placeid = []
pname = []
region = []
for i in row_places:
    placeid.append(i[2])
    pname.append(i[3])
    region.append(i[4])
print(placeid)
print(pname)
print(len(placeid),len(pname),len(region))

[]
[]
0 0 0


In [631]:
for i in range(len(placeid)):   # places 테이블
    sql = "insert into tour values('" + placeid[i] + "', 0, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL, NULL)"
    try:
        cur.execute(sql)
        con.commit()
    except:
        con.rollback()
    
con.close()

In [36]:
# 오류처리
f = open('추가.txt','r',encoding='utf8')
data = f.read().splitlines()
        
placeID = []
placename = []
tourname = []
for i in data:
    if i == '':
        continue
    else:
        #print(i)
        place = re.sub("\*",'',i)
        places = place.split(' $ ')[0]
        #print(place.split('$')[1])
        tourname.append(place.split(' $ ')[1])
        placeID.append(places.split('/')[0])
        placename.append(places.split('/')[1])
        #print(place.split(' $ ')[1],places.split('/')[0],places.split('/')[1])
        if place.split(' $ ')[1] == '':
            print(i)
        #print(places.split('/'))
#print(tourname[20],'#', tourname[21],'#', tourname[29],'#', tourname[130])
print(tourname[15],tourname[53],tourname[24], tourname[1])

 칼리오페 강진시장 (4, 9일) 대릉갈비 오크밸리 3D 라이팅쇼 '소나타오브라이트'


In [4]:
def search_keyword(api_key_decode,keyword):
    url = f'https://apis.data.go.kr/B551011/KorService/searchKeyword?serviceKey={api_key_decode}&numOfRows=10&pageNo=1&MobileOS=ETC&MobileApp=NEXPOT&_type=json&listYN=Y&arrange=B&keyword={keyword}'
    result = requests.get(url, verify=False)
    data = json.loads(result.text)
    if data['response']['body']['numOfRows'] == 0:
        return
    keywords = data['response']['body']['items']['item']
    for i in keywords:
        if i['title'] == keyword:
            keyword_data  = i
            break
        else :
            keyword_data = data['response']['body']['items']['item'][0]
    contentid = keyword_data['contentid']
    contenttypeid = keyword_data['contenttypeid']
    title = keyword_data['title']
    return [contentid, title, contenttypeid]

In [5]:
def detail_common(api_key_decode,contentid,title):
    url = f'https://apis.data.go.kr/B551011/KorService/detailCommon?MobileOS=ETC&MobileApp=NEXPOT&serviceKey={api_key_decode}&_type=json&contentId={contentid}&defaultYN=Y&addrinfoYN=Y&overviewYN=Y'
    result = requests.get(url, verify=False)
    data = json.loads(result.text)
    content_data = data['response']['body']['items']['item'][0]
    addr = content_data['addr1']
    tel = content_data['tel']
    overview = re.sub("'", "",content_data['overview'])
    return [addr, tel, overview]


In [19]:
def detail_intro(api_key_decode,contentid,contenttypeid):
    url = f'https://apis.data.go.kr/B551011/KorService/detailIntro?MobileOS=ETC&MobileApp=NEXPOT&serviceKey={api_key_decode}&_type=json&contentId={contentid}&contentTypeId={contenttypeid}'
    result = requests.get(url, verify=False)
    data = json.loads(result.text)
    intro_data = data['response']['body']['items']['item'][0]
    infocenter= None
    restdate= None
    usetime= None
    parking= None
    chkbabycarriage= None
    chkpet= None
    chkcreditcard = None
    for data in intro_data.keys():
        if data.startswith('infocenter'):
            infocenter = intro_data[data]
        if data.startswith('restdate'):
            restdate = intro_data[data]
        if data.startswith('usetime') or data.startswith('opentime'):
            usetime = intro_data[data]
        if data.startswith('parking'):
            parking = intro_data[data]
        if data.startswith('chkbabycarriage'):
            chkbabycarriage = intro_data[data]
        if data.startswith('chkpet'):
            chkpet = intro_data[data]
        if data.startswith('chkcreditcard'):
            chkcreditcard = intro_data[data]
    return [infocenter,restdate,usetime,re.sub("'","",parking),chkbabycarriage,chkpet,chkcreditcard]

In [20]:
def main():
    #servicekey = 'JaSZeDTHJ5hQVERoJVnKpg%2F5ff8fMXkU%2F2aubsJl%2Fl9lxDA5ZnIEz0VuOP0cpiwYRX1bFDS0yWX7XbRd0NFg7w%3D%3D'
    api_key_decode = 'JaSZeDTHJ5hQVERoJVnKpg%2F5ff8fMXkU%2F2aubsJl%2Fl9lxDA5ZnIEz0VuOP0cpiwYRX1bFDS0yWX7XbRd0NFg7w%3D%3D'
    #api_key_decode = unquote(servicekey)
    local = 'nexpot.c43ypw0liaa2.ap-northeast-2.rds.amazonaws.com'  # aws rds 접근
    con = db.connect(
        host=local,
        user='admin',
        db='Nexpot',
        password='qwertyuiop123',
        charset='utf8mb4'
    )
    cur = con.cursor()
    cur.execute("select * from places where placeID not in (select tour.placeID from places, tour where places.placeID = tour.placeID);")
    # DB에 없는 장소 갖고오기
    row_places = cur.fetchall()
    placeid = []
    pname = []
    region = []
    for i in row_places:
        placeid.append(i[2])
        pname.append(re.sub('&','',i[3]))
        region.append(i[4])
    
    df = pd.DataFrame(columns=range(14))
    '''
    df = pd.DataFrame(columns=range(14))
    f = open('추가.txt','r',encoding='utf8')
    # 오류처리
    data = f.read().splitlines()
    placeID = []
    placename = []
    tourname = []
    for i in data:
        if i == '':
            continue
        else:
            place = re.sub("\*",'',i)
            place = re.sub('&','',place)
            places = place.split(' $ ')[0]
            tourname.append(place.split(' $ ')[1])
            placeID.append(places.split('/')[0])
            placename.append(places.split('/')[1])
    '''
    for k in range(len(placeid)):   #len(pname)
        print(k)
        id = placeid[k]
        name = pname[k]
        
        contit = search_keyword(api_key_decode,name)
        if contit == None:
            df.loc[len(df)] = [id,0,None,None,None,None,None,None,None,None,None,None,None,None]
            continue
        addtelove= detail_common(api_key_decode, contit[0],contit[1])
        #region1 = region[k][:2]
        #if region1[0] not in addtelove[0] and region1[1] not in addtelove[0]:
        #    df.loc[len(df)] = [id,0,None,None,None,None,None,None,None,None,None,None,None,None]
        #    continue
        irupccc = detail_intro(api_key_decode,contit[0],contit[2])
        df.loc[len(df)] = [id,1]+contit[:2]+addtelove+irupccc
        print([id,1]+contit[:2]+addtelove+irupccc)

    df.columns = ['placeID','exist','contentid','title','addr','tel','overview',
                  'infocenter','restdate','usetime','parking','chkbabycarriage','chkpet','chkcreditcard']
    print(df)
    for i in range(len(df)):
        
        sql = "insert into tour values('" + str(df['placeID'][i]) + "','" +  str(df['exist'][i]) + "','" + \
        str(df['contentid'][i]) + "','" + str(df['title'][i]) + "','" + str(df['addr'][i]) + "','" + \
        str(df['tel'][i]) + "','" + str(df['overview'][i]) + "','" + str(df['infocenter'][i]) + "','" + \
        str(df['restdate'][i]) + "','" + str(df['usetime'][i]) + "','" + str(df['parking'][i]) + "','" + \
        str(df['chkbabycarriage'][i]) + "','" + str(df['chkpet'][i]) + "','" + str(df['chkcreditcard'][i]) +"')"
        '''
        sql = "UPDATE tour SET exist = '" +  str(df['exist'][i]) + "', contentid = '" + str(df['contentid'][i]) + \
         "', title = '" + str(df['title'][i]) + "', addr = '" + str(df['addr'][i]) + "', tel = '" + \
        str(df['tel'][i]) + "', overview = '" + str(df['overview'][i]) + "', infocenter = '" + str(df['infocenter'][i]) + \
        "',restdate = '" + str(df['restdate'][i]) + "', usetime = '" + str(df['usetime'][i]) + "', parking = '" + str(df['parking'][i]) \
        + "', chkbabycarriage = '" + str(df['chkbabycarriage'][i]) + "', chkpet = '" + str(df['chkpet'][i]) + "', chkcreditcard = '" + \
        str(df['chkcreditcard'][i]) +"' where placeID = '" + str(df['placeID'][i]) + "';"
        '''
        try:
            cur.execute(sql)
            con.commit()
            print('good')
        except:
            con.rollback()
    
    con.close()

In [21]:
main()

0


C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(
C:\Users\USER\anaconda3\lib\site-packages\urllib3\connectionpool.py:981: InsecureRequestWarning: Unverified HTTPS request is being made to host 'apis.data.go.kr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn(


['17027131', 1, '2751857', '의림지솔밭공원', '충청북도 제천시 의림지로 33', '', '의림지솔밭공원은 제천 제1경인 의림지에서 용두산 쪽으로 도로를 따라 올라가 제2의림지인 비룡담저수지 둑 아래에 소나무 숲이 우거진 공원이다. 이리저리 휘며 자란 소나무들의 독특한 아름다움이 빼어난 공원으로, 숲속에 인공 물길을 만들어 물소리와 새소리를 정겹게 감상하면서 휴식할 수 있는 공간이다. 1980년대 제천시 승격 이후, 제천 의용소방대에 의해 아카시아와 돌밭의 척박한 환경에서 자라난 소나무를 의림지와 연계해서 제천의 명물로 탄생시킨 공원이다. 잔디마당을 비롯한 솔숲에 물길을 만들어 솔숲 길을 산책하기 좋아지도록 하여 시민들의 휴식처 및 각 학교(어린이집 등)들의 소풍 장소 등으로 활용되고 있다. 2020년 솔숲 내 자연형 인공수로를 만들어 여름철 물놀이를 즐길 수 있도록 하였다.', '043-651-7101', '', '연중 상시 개방', '가능 (소형차 약 50~70대 정도, 충혼탑으로 가기 전 공원부지 옆 도로에 일렬 주차 가능, 꼬네 카페 앞 주차선 표시된 구간 등)', '없음', '가능', '없음']
    placeID exist contentid    title              addr tel  \
0  17027131     1   2751857  의림지솔밭공원  충청북도 제천시 의림지로 33       

                                            overview    infocenter restdate  \
0  의림지솔밭공원은 제천 제1경인 의림지에서 용두산 쪽으로 도로를 따라 올라가 제2의림...  043-651-7101            

    usetime                                            parking  \
0  연중 상시 개방  가능 (소형차 약 50~70대 정도, 충혼탑으로 가기 전 공원부지 옆 도로에 일렬 ...   

 